In [1]:
from collections import namedtuple
from itertools import product
from functools import reduce
from operator import mul
import gurobipy as grb
GRB = grb.GRB

In [2]:
CellDef = namedtuple('CellDef', 'cell_list formula')

## Add variables for each square

In [3]:
def add_variables(m, size):
    v = {}
    for i in range(1, size+1):
        for j in range(1, size+1):
            v[(i,j)] = m.addVar(lb=1, ub=size, vtype=GRB.INTEGER, name='v'+str(i)+str(j))
            for k in range(1, size+1):
                v[(i,j,k)] = m.addVar(lb=0, ub=1, vtype=GRB.BINARY, name='vi'+str(i)+str(j)+str(k))
    return v

## Add basic constraints

In [4]:
def add_basic_constraints(m, v, size):
    # Connect indicator vars to cell variables
    for i in range(1, size+1):
        for j in range(1, size+1):
            sqr_expr = 0
            sum_ind = 0
            for k in range(1, size+1):
                sqr_expr += k * v[(i,j,k)]
                sum_ind += v[(i,j,k)]
            m.addConstr(v[(i,j)], GRB.EQUAL, sqr_expr, name='cell_val'+str(i)+str(j))
            m.addConstr(sum_ind, GRB.EQUAL, 1, name='cell_ind'+str(i)+str(j))

    # Each row has one of each digit
    for i in range(1, size+1):
        for k in range(1, size+1):
            row_expr = 0
            for j in range(1, size+1):
                row_expr += v[(i,j,k)]
            m.addConstr(row_expr, GRB.EQUAL, 1, name='row'+str(i)+'_val'+str(k))

    # Each column has 1 of each digit
    for j in range(1, size+1):
        for k in range(1, size+1):
            col_expr = 0
            for i in range(1, size+1):
                col_expr += v[(i,j,k)]
            m.addConstr(col_expr, GRB.EQUAL, 1, name='col'+str(j)+'_val'+str(k))

## Kenken cell constraints

In [5]:
def extract_int(formula):
    if formula[-1].isdigit():
        return int(formula)
    else:
        return int(formula[:-1])

In [6]:
def plus_constrs(m, v, cell_list, num, namestr):
    expr = 0
    for (i, j) in cell_list:
        expr += v[(i,j)]
    m.addConstr(expr, GRB.EQUAL, num, name='plus' + str(num) + namestr)

In [7]:
def find_next(size, n, num):
    """
    Yield all lists of n integers between 1 and size that multiply to num
    """
    for p in product(range(1, size+1), repeat=n):
        if reduce(mul, p, 1) == num:
            yield(p)
    return
        
def times_constrs(m, v, cell_list, num, namestr):
    varlist = []
    for mult_list in find_next(size, len(cell_list), num):
        mult_name = str(mult_list).replace(' ','')
        varlist.append(m.addVar(lb=0, ub=1, vtype=GRB.BINARY, name='mult_and'+namestr + mult_name))
        vlist = []
        for ((i,j), val) in zip(cell_list, mult_list):
            vlist.append(v[(i,j,val)])
        m.addGenConstrAnd(varlist[-1], vlist)
    vor = m.addVar(lb=0, ub=1, vtype=GRB.BINARY, name='mul'+namestr)
    m.addGenConstrOr(vor, varlist)

In [8]:
def minus_constrs(m, v, cell_list, num, namestr):
    if len(cell_list) != 2:
        print('Illegal minus cell: {}'.format(cell_list))
        return
    v1 = v[(cell_list[0][0], cell_list[0][1])]
    v2 = v[(cell_list[1][0], cell_list[1][1])]
    mv = m.addVar(lb=-size, ub=size, vtype=GRB.INTEGER, name='mv'+namestr)
    av = m.addVar(lb=0, ub=size, vtype=GRB.INTEGER, name='rv'+namestr)
    m.addGenConstrAbs(av, mv, name='minus'+namestr)
    m.addConstr(mv, GRB.EQUAL, v1 - v2, name='minus' + str(num) + namestr)
    m.addConstr(av, GRB.EQUAL, num, name='minus_abs' + str(num) + namestr)

In [9]:
def divide_constrs(m, v, cell_list, num, namestr):
    if len(cell_list) != 2:
        print('Illegal minus cell: {}'.format(cell_list))
        return
    (i1, j1) = (cell_list[0][0], cell_list[0][1])
    (i2, j2) = (cell_list[1][0], cell_list[1][1])
    varlist = []
    for d1 in range(1, size // num + 1):
        d2 = d1 * num
        varlist.append(m.addVar(lb=0, ub=1, vtype=GRB.BINARY, name='div'+namestr + str(d1) + str(d2)))
        m.addGenConstrAnd(varlist[-1], [v[(i1, j1, d1)], v[(i2, j2, d2)]])
        varlist.append(m.addVar(lb=0, ub=1, vtype=GRB.BINARY, name='div'+namestr + str(d2) + str(d1)))
        m.addGenConstrAnd(varlist[-1], [v[(i1, j1, d2)], v[(i2, j2, d1)]])
    vor = m.addVar(lb=0, ub=1, vtype=GRB.BINARY, name='div'+namestr)
    m.addGenConstrOr(vor, varlist)
    return

In [10]:
def singleton(m, v, cell_list, num, namestr):
    i = cell_list[0][0]
    j = cell_list[0][1]
    m.addConstr(v[(i,j)], GRB.EQUAL, num, name='single' + str(num) + namestr)

In [11]:
op_method = {
    '+': plus_constrs,
    '*': times_constrs,
    'x': times_constrs,
    '-': minus_constrs,
    '/': divide_constrs,
    's': singleton
}

In [12]:
def add_cell_constraints(m, v, cells):
    for cell in cells:
        ncell = CellDef(*cell)
        formula = ncell.formula
        num = extract_int(formula)
        op = formula[-1] if len(formula) > 1 else 's'
        cell_list = ncell.cell_list
        namestr = str(cell_list).replace(' ','')
        print('Forming {} constraints for squares {}'.format(formula, namestr))
        op_method[op](m, v, cell_list, num, namestr)
    print()

## Show square

In [29]:
def show_square(size, v):
    print('\nSolution:\n')
    for i in range(1, size+1):
        for j in range(1, size+1):
            print(round(v[(i,j)].x), ' ', end='')
        print()

In [30]:
def solve_kenken(size, cells):
    m = grb.Model()
    v = add_variables(m, size)
    add_basic_constraints(m, v, size)
    add_cell_constraints(m, v, cells)
    m.write('kenken.lp')
    m.optimize()
    show_square(size, v)

In [15]:
size = 5
cells = [
    ([(1,1), (1,2)], '2/'),
    ([(1,3), (1,4), (1,4)], '60x'),
    ([(2,1), (3,1)], '5+'),
    ([(2,2), (3,2)], '2-'),
    ([(2,5)], '2'),
    ([(3,3), (4,3), (4,4)], '7+'),
    ([(3,4), (3,5)], '4-'),
    ([(4,1), (4,2)], '1-'),
    ([(4,5), (5,5)], '2-'),
    ([(5,1)], '4'),
    ([(5,2), (5,3), (5,4)], '10+')
]

In [16]:
solve_kenken(5, cells)

Forming 2/ constraints for squares [(1,1),(1,2)]
Forming 60x constraints for squares [(1,3),(1,4),(1,4)]
Forming 5+ constraints for squares [(2,1),(3,1)]
Forming 2- constraints for squares [(2,2),(3,2)]
Forming 2 constraints for squares [(2,5)]
Forming 7+ constraints for squares [(3,3),(4,3),(4,4)]
Forming 4- constraints for squares [(3,4),(3,5)]
Forming 1- constraints for squares [(4,1),(4,2)]
Forming 2- constraints for squares [(4,5),(5,5)]
Forming 4 constraints for squares [(5,1)]
Forming 10+ constraints for squares [(5,2),(5,3),(5,4)]

Optimize a model with 113 rows, 170 columns and 551 nonzeros
Model has 16 general constraints
Variable types: 0 continuous, 170 integer (137 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 5e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 113 rows and 170 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in

In [17]:
size = 7
cells = [
    ([(1,1), (1,2), (1,3)], '140x'),
    ([(1,4), (2,4)], '2/'),
    ([(1,5), (2,5), (2,6)], '8+'),
    ([(1,6), (1,7)], '5-'),
    ([(2,1), (2,2)], '2-'),
    ([(2,3)], '7'),
    ([(2,7), (3,7)], '3-'),
    ([(3,1), (4,1)], '1-'),
    ([(5,1), (6,1)], '2-'),
    ([(3,2), (3,3)], '7+'),
    ([(4,2), (4,3)], '6x'),
    ([(5,2), (5,3)], '2/'),
    ([(6,2), (6,3)], '1-'),
    ([(7,1), (7,2)], '5-'),
    ([(3,4), (4,4)], '1-'),
    ([(5,4)], '3'),
    ([(3,5), (3,6), (4,5), (5,5)], '840x'),
    ([(4,6), (5,6)], '11+'),
    ([(4,7), (5,7)], '21x'),
    ([(6,4), (7,3), (7,4)], '14+'),
    ([(6,5), (7,5), (7,6)], '12+'),
    ([(6,6)], '2'),
    ([(6,7), (7,7)], '5+')
]

In [18]:
solve_kenken(size, cells)

Forming 140x constraints for squares [(1,1),(1,2),(1,3)]
Forming 2/ constraints for squares [(1,4),(2,4)]
Forming 8+ constraints for squares [(1,5),(2,5),(2,6)]
Forming 5- constraints for squares [(1,6),(1,7)]
Forming 2- constraints for squares [(2,1),(2,2)]
Forming 7 constraints for squares [(2,3)]
Forming 3- constraints for squares [(2,7),(3,7)]
Forming 1- constraints for squares [(3,1),(4,1)]
Forming 2- constraints for squares [(5,1),(6,1)]
Forming 7+ constraints for squares [(3,2),(3,3)]
Forming 6x constraints for squares [(4,2),(4,3)]
Forming 2/ constraints for squares [(5,2),(5,3)]
Forming 1- constraints for squares [(6,2),(6,3)]
Forming 5- constraints for squares [(7,1),(7,2)]
Forming 1- constraints for squares [(3,4),(4,4)]
Forming 3 constraints for squares [(5,4)]
Forming 840x constraints for squares [(3,5),(3,6),(4,5),(5,5)]
Forming 11+ constraints for squares [(4,6),(5,6)]
Forming 21x constraints for squares [(4,7),(5,7)]
Forming 14+ constraints for squares [(6,4),(7,3),(7,4

In [19]:
size = 5
cells = [
    ([(1,1)], '3'),
    ([(1,2), (2,2)], '6+'),
    ([(1,3), (2,3)], '9+'),
    ([(1,4), (2,4)], '2/'),
    ([(1,5), (2,5)], '4+'),
    ([(2,1), (3,1)], '1-'),
    ([(3,2)], '2'),
    ([(3,5), (4,5)], '1-'),
    ([(4,1), (5,1)], '1-'),
    ([(4,2), (5,2)], '1-'),
    ([(4,3), (4,4)], '1-'),
    ([(5,4), (5,5)], '6x'),
    ([(5,3)], '1')
]

In [20]:
solve_kenken(size, cells)

Forming 3 constraints for squares [(1,1)]
Forming 6+ constraints for squares [(1,2),(2,2)]
Forming 9+ constraints for squares [(1,3),(2,3)]
Forming 2/ constraints for squares [(1,4),(2,4)]
Forming 4+ constraints for squares [(1,5),(2,5)]
Forming 1- constraints for squares [(2,1),(3,1)]
Forming 2 constraints for squares [(3,2)]
Forming 1- constraints for squares [(3,5),(4,5)]
Forming 1- constraints for squares [(4,1),(5,1)]
Forming 1- constraints for squares [(4,2),(5,2)]
Forming 1- constraints for squares [(4,3),(4,4)]
Forming 6x constraints for squares [(5,4),(5,5)]
Forming 1 constraints for squares [(5,3)]

Optimize a model with 116 rows, 168 columns and 554 nonzeros
Model has 13 general constraints
Variable types: 0 continuous, 168 integer (133 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 5e+00]
  RHS range        [1e+00, 9e+00]
Presolve removed 116 rows and 168 columns
Presolve time: 0.00s
Presolve: A

In [23]:
size = 7
cells = [
    ([(1,1), (2,1)], '20x'),
    ([(1,2), (2,2)], '2-'),
    ([(1,3), (1,4)], '10+'),
    ([(2,3), (2,4)], '8+'),
    ([(1,5), (2,5)], '3-'),
    ([(1,6), (1,7)], '2/'),
    ([(3,1), (4,1)], '3/'),
    ([(3,2), (3,3)], '2/'),
    ([(4,2), (4,3)], '2-'),
    ([(3,4), (3,5)], '1-'),
    ([(2,6), (2,7), (3,6)], '10+'),
    ([(3,7)], '7'),
    ([(4,4), (4,5), (5,4), (5,5)], '70x'),
    ([(4,6), (5,6)], '7+'),
    ([(4,7), (5,7), (6,7)], '12+'),
    ([(5,1), (6,1)], '6-'),
    ([(5,2), (6,2)], '10+'),
    ([(5,3), (6,3)], '2/'),
    ([(7,1), (7,2)], '5+'),
    ([(6,4), (7,4)], '2/'),
    ([(6,5), (7,5)], '2-'),
    ([(7,6), (7,7)], '1-'),
    ([(6,6)], '4'),
    ([(7,3)], '1')
]

In [31]:
solve_kenken(size, cells)

Forming 20x constraints for squares [(1,1),(2,1)]
Forming 2- constraints for squares [(1,2),(2,2)]
Forming 10+ constraints for squares [(1,3),(1,4)]
Forming 8+ constraints for squares [(2,3),(2,4)]
Forming 3- constraints for squares [(1,5),(2,5)]
Forming 2/ constraints for squares [(1,6),(1,7)]
Forming 3/ constraints for squares [(3,1),(4,1)]
Forming 2/ constraints for squares [(3,2),(3,3)]
Forming 2- constraints for squares [(4,2),(4,3)]
Forming 1- constraints for squares [(3,4),(3,5)]
Forming 10+ constraints for squares [(2,6),(2,7),(3,6)]
Forming 7 constraints for squares [(3,7)]
Forming 70x constraints for squares [(4,4),(4,5),(5,4),(5,5)]
Forming 7+ constraints for squares [(4,6),(5,6)]
Forming 12+ constraints for squares [(4,7),(5,7),(6,7)]
Forming 6- constraints for squares [(5,1),(6,1)]
Forming 10+ constraints for squares [(5,2),(6,2)]
Forming 2/ constraints for squares [(5,3),(6,3)]
Forming 5+ constraints for squares [(7,1),(7,2)]
Forming 2/ constraints for squares [(6,4),(7,4